In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, K):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数
    #data = data[['date','stock','close','turnover_sum','change']]
    #data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'turnover_sum', 'change', 'change_sum', 'turnover_N1', 'turnover_N2', 'change_N'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        turn_o = starter['turnover_sum']
        change_s = starter['change_sum']
        differ_t = turn_o.values[1]-turn_o.values[0]                  ###计算T前N1天的成交金额总量
        differ_t2 = turn_o.values[3]-turn_o.values[1]                 ###计算T后N2天的成交金额总量
        differ_c = change_s.values[3]-change_s.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(6,'turnover_N1',[differ_t])
        stock_a.insert(7,'turnover_N2',[differ_t2])
        stock_a.insert(8,'change_N',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)

    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date','stock','close','turnover_N1','turnover_N2','change_N']]
    
    stock_Tmax = stock_f.sort_values(by = ['turnover_N1'], ascending = False).head(K)     ###取出N天成交金额最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change_N'], ascending = False).head(K)         ###取出T日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf    

In [4]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017/2017.csv',encoding='gbk',engine='python')

In [5]:
#stock_m = max_stock(data)

stock_eg = data[data['stock'] == 1]

In [6]:
stock_bestinf = best_perf(data, stock_eg, '2017-11-24', 55, 10, 100)

In [7]:
stock_bestinf

,date,stock,close,turnover_N1,turnover_N2,change_N
93384,2017-11-24,830,13.70,3.810954e+10,1.217803e+10,0.014580
264368,2017-11-24,2600,9.65,3.957565e+10,2.731000e+09,0.012815
547020,2017-11-24,300355,12.21,5.273424e+10,8.829097e+09,0.009362
665987,2017-11-24,600460,11.30,5.214161e+10,2.254029e+10,0.020579
